<a href="https://colab.research.google.com/github/GokayToga/FloodNet_Challange-GT/blob/main/FloodNet_Challange_GT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To Do:

*   resize the images to 713x713 ✅
*   Augment the images with random shuffling, scal- ing, flipping, and random rotation ❓
*  First Build basic U-net to experiment
*  Then PSPNet, ENet, and DeepLabv3+ can be implemented and experimented on.




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Libraries**

In [6]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import cv2
import imageio

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import ConfusionMatrixDisplay
from collections import Counter

from tensorflow.image import resize as tf_resize
from tensorflow import io as tf_io
from tensorflow import image as tf_image

from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Data Resizing and Augmentation

In [4]:
RESIZE = (713,713) #shape of resized image
drive_path = "/content/drive/MyDrive/FloodNet-Supervised_v1.0"
local_path = "/content/Augmented_Data"

def Augment_Resize(path, resize=RESIZE):
  #used_path = os.path.join(drive_path, path) #path for diffent datasets
  used_path = drive_path + path
  save_format = os.path.splitext(used_path)[1].lower()

  Aug_parameters = ImageDataGenerator(   #augmentation parameters
      rescale=1./255,          # Scaling
      horizontal_flip=True,    # Random horizontal flip
      vertical_flip=True,      # Random vertical flip
      rotation_range=45        # Random rotation in the range [-45, 45] degrees
  )

  if len(os.listdir(used_path)) > 1:

    length = len(os.listdir(used_path))
    #print( length )

    for img_name in tqdm(os.listdir(used_path)):
      img_path = used_path + img_name
      # Read and resize the image
      img = cv2.imread(img_path)

      # Check if the image was read successfully
      if img is not None:
        img = cv2.resize(img, resize)

        # Reshape to meet the requirements of ImageDataGenerator
        img = img.reshape((1,) + img.shape)

        # Generate augmented images
        for batch in Aug_parameters.flow(img, batch_size=1, save_to_dir=local_path, save_prefix=img_name.split('.')[0], save_format=save_format):
          break  # Stop after one augmented image (to avoid infinite loop)
      else:
        print(f"Error: Unable to read image at {img_path}")
  else:
    print(f"{path}  images are already saved")


# Data resizer :
- (Because of the problems in augmentation we countinue with only resizing)

In [41]:


RESIZE = (713,713) #shape of resized image
drive_path = "/content/drive/MyDrive/FloodNet-Supervised_v1.0"
local_path = "/content/Augmented_Data"

def Augment_Resize2(path, resize=RESIZE):
  #used_path = os.path.join(drive_path, path) #path for diffent datasets
  used_path = drive_path + path
  save_format = os.path.splitext(used_path)[1].lower()


  if len(os.listdir(used_path)) > 1:

    length = len(os.listdir(used_path))
    #print( length )

    for img_name in tqdm(os.listdir(used_path)):
      img_path = used_path + "/" + img_name #os.path.join(used_path, img_name)
      save_path = local_path + path + "/" + img_name
      #print(f"Attempting to read image at: {img_path}")

      try:
          img = imageio.imread(img_path)
      except Exception as e:
          print(f"Error: {e}")
          continue

      # Check if the image was read successfully
      if img is not None:

        img = cv2.resize(img, resize)
        imageio.imwrite(save_path, img)

      else:
        print(f"Error: Unable to read image at {img_path}")
  else:
    print(f"{path}  images are already saved")


In [ ]:
#Test
Augment_Resize2("/test/test-label-img")
Augment_Resize2("/test/test-org-img")

#Train
Augment_Resize2("/train/train-label-img")
Augment_Resize2("/train/train-org-img")

#Val
Augment_Resize2("/val/val-label-img")
Augment_Resize2("/val/val-org-img")



## Model: